# Introduction

This project focuses on building an end to end NLP pipeling to analyse the customer's reviews in British airline. Through this NLP pipeline I will use mulitple machine learning models either traditional way or deep learning one, to determine the most accurate and effective one.

# Problem Statement

In the airline Industry, the customer satisfaction plays an important role in business, and the airline reputation. One of the key indicators about any airline specially here the British airline is whether the customers would recommend our airline or not. So based on customers feedback, our airline will have a good income, reputation, etc. but also the customer feedback is so important to know which points we have lack at and so we could imporve our services.

As a data scientest at British airline I will analyze customer reviews so we can predict their recommendations. By automating this prediction, the airline can do some improvments and recommendation leading to a better customer satisfiction.

# Reference for the dataset

https://www.kaggle.com/datasets/dharun4772/british-airline-review-dataset

# Import Libraries

In [ ]:
import pandas as pd
import nltk
import spacy
!pip install textacy
import textacy
nlp = spacy.load("en_core_web_sm")
pd.set_option("max_colwidth", None)
import sklearn.metrics.pairwise
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
import sklearn.ensemble
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
from sklearn.metrics import  accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import random
import transformers
!pip install datasets
import datasets
import matplotlib.pyplot as plt
import numpy as np

/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


Imported the needed libraries in my code

# Load the dataset

In [ ]:
df = pd.read_csv("/content/airline_review.csv")

Loaded the dataset that I will work on it.

# Exploratory Data Anaylsis (EDA)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3616 entries, 0 to 3615
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Unnamed: 0           3616 non-null   int64 
 1   rating               3616 non-null   int64 
 2   header               3616 non-null   object
 3   author               3616 non-null   object
 4   date                 3616 non-null   object
 5   place                3616 non-null   object
 6   content              3616 non-null   object
 7   aircraft             1902 non-null   object
 8   traveller_type       2895 non-null   object
 9   seat_type            3614 non-null   object
 10  route                2891 non-null   object
 11  date_flown           2888 non-null   object
 12  seat_comfort         3616 non-null   int64 
 13  cabin_staff_service  3616 non-null   int64 
 14  food_beverages       3616 non-null   int64 
 15  ground_service       3616 non-null   int64 
 16  value_

To know the number of enteries (rows and columns)

In [ ]:
df.columns

Index(['Unnamed: 0', 'rating', 'header', 'author', 'date', 'place', 'content',
       'aircraft', 'traveller_type', 'seat_type', 'route', 'date_flown',
       'seat_comfort', 'cabin_staff_service', 'food_beverages',
       'ground_service', 'value_for_money', 'recommended', 'entertainment',
       'trip_verified'],
      dtype='object')

To check the list of columns in the dataset

In [ ]:
df = df.drop_duplicates()

To remove all duplicates in the dataset

In [ ]:
df.isnull().sum()

,0
Unnamed: 0,0
rating,0
header,0
author,0
date,0
place,0
content,0
aircraft,1714
traveller_type,721
seat_type,2


To check for any missing values

In [ ]:
df = df.dropna()

To remove any missing rows

# Data Preprocessing

In [ ]:
df = df.drop(columns=['Unnamed: 0'], errors='ignore')

To remove the unnamed column

In [ ]:
df.columns

Index(['rating', 'header', 'author', 'date', 'place', 'content', 'aircraft',
       'traveller_type', 'seat_type', 'route', 'date_flown', 'seat_comfort',
       'cabin_staff_service', 'food_beverages', 'ground_service',
       'value_for_money', 'recommended', 'entertainment', 'trip_verified'],
      dtype='object')

In [ ]:
df = df[["content", "recommended"]]
df.columns = ["text", "label"]
le = sklearn.preprocessing.LabelEncoder()
df["label"] = le.fit_transform(df["label"])
number_of_classes = df["label"].nunique()
df.head()

,text,label
0,"Just returned from Chicago, flew out 10 days ago on American Airlines absolutely superb in every way, had high expectations on return flight with BA. What a disappointment. The Airbus A380 may be nice from a pilots perspective but as a passenger it was awful. Very uncomfortable seats, the inflight entertainment and flight tracker failed to work throughout the flight, the inflight meal was inedible and the service was mediocre at best. Our short flight from Heathrow to Manchester was much improved, very welcoming and attentive in flight staff and the flight even arrived early. In future we will travel with one of the American carriers.",0
1,"BA standards continue to decline every time I fly with them. This time, a 45 min late departure, which seems the norm for BA, and no information from the rude and clueless boarding gate staff. The Club lounge was overcrowded, dirty and grubby and cleaning staff could not cope. How many more cutbacks is BA going to apply before they become truly low cost. As a 35 year veteran user of BA, while people complained of Alex Cruz cutting costs, the past 3 years has seen them decline very quickly under their latest CEO. One reads the occasional article about BA improving things, but I have yet to see anything substantive. There seem to be few British staff left amongst the cabin crew, so maybe time for a name change and give the flag carrier status to a proper low-cost - and sadly the staff service has declined year on year. Onboard, the catering remains poor and surprised that Do&Co can produce such rubbish meals when I have sampled their Turkish Airlines equivalent in August that was superb - guessing BA are too tight with their budget. Onboard WiFi service is a joke and should be avoided.",0
2,"Awful. Business class check in queue just as long as for economy, probably because half of the desks were not staffed and Terminal 5 is chaotic. Business lounge overfull, couldn't get seats. Delayed flight, unmanaged chaos at the gate. No drink on boarding, first drink served an hour after take-off. The meal was poor, used to get better in economy on other airlines. British Airways has won the race to the bottom of the cheapskate stakes. Not looking forward to the flight home.",0
3,"Not a reliable airline. You cannot trust the timing at all. I had a 6.45am flight, boarding was on time, but then we are sitting in the plane for already (1.5 hours) and the expected departure is still not clear. They have discovered the engine is not working! I am risking to be late for a business meeting in Geneva. How come they bring a not working plane to the gate. The funny thing is that I had another business trip in spring 2023, also with BA to Amsterdam. When I arrived to the Airport that time - I was told I don't have a seat. I was put on the next flight in several hours, which should have get me to the business meeting late, but still not too much. A lot of stress. In the end, on the second plane we were sitting in the cabin for 1.5 hours and I have missed the meeting. I thought it was a dramatic unlucky case that time, but today I see it's routine for BA not to fly on time.",0
6,"The airplanes and the lounges are worn out, old and broken. From Dallas to Heathrow, multiple first class seating and electronics were non functional and poorly designed. The first class seating from Heathrow to Dubrovnik was nothing more than a tray blocking the middle seat of a standard coach ticket. Very disappointed.",0


I started by the relevant columns: Distributed into 2 parts (preprocessed- label) which are (content- recommended), then I renamed as(text- label) to make it easier working with the dataset. then I created the LabelEncoder to convert all labels from categorical(text) into numerical values. As for "no" will be 0 and for "yes" will be 1 (in recommendations).

In [ ]:
from os import remove
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
def remove_irrelevant_chars(text):
    return ''.join(char for char in text if char.isalnum() or char.isspace())
def to_lowercase(text):
    return text.lower()
def remove_punctuation(text):
    return ''.join(char for char in text if char not in string.punctuation)
def remove_stop_words(tokens):
    stop_words = set(stopwords.words('english'))
    return [word for word in tokens if word not in stop_words]
def lemmatize_words(tokens):
    return [lemmatizer.lemmatize(word) for word in tokens]
def clean_text(text):
    text = remove_irrelevant_chars(text)
    text = to_lowercase(text)
    text = remove_punctuation(text)
    tokens = word_tokenize(text)
    tokens = remove_stop_words(tokens)
    tokens = lemmatize_words(tokens)
    text = " ".join(tokens)
    return text

df['cleaned_text'] = df['text'].apply(clean_text)

df[['text', 'cleaned_text']].head()

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,text,cleaned_text
0,"Just returned from Chicago, flew out 10 days ago on American Airlines absolutely superb in every way, had high expectations on return flight with BA. What a disappointment. The Airbus A380 may be nice from a pilots perspective but as a passenger it was awful. Very uncomfortable seats, the inflight entertainment and flight tracker failed to work throughout the flight, the inflight meal was inedible and the service was mediocre at best. Our short flight from Heathrow to Manchester was much improved, very welcoming and attentive in flight staff and the flight even arrived early. In future we will travel with one of the American carriers.",returned chicago flew 10 day ago american airline absolutely superb every way high expectation return flight ba disappointment airbus a380 may nice pilot perspective passenger awful uncomfortable seat inflight entertainment flight tracker failed work throughout flight inflight meal inedible service mediocre best short flight heathrow manchester much improved welcoming attentive flight staff flight even arrived early future travel one american carrier
1,"BA standards continue to decline every time I fly with them. This time, a 45 min late departure, which seems the norm for BA, and no information from the rude and clueless boarding gate staff. The Club lounge was overcrowded, dirty and grubby and cleaning staff could not cope. How many more cutbacks is BA going to apply before they become truly low cost. As a 35 year veteran user of BA, while people complained of Alex Cruz cutting costs, the past 3 years has seen them decline very quickly under their latest CEO. One reads the occasional article about BA improving things, but I have yet to see anything substantive. There seem to be few British staff left amongst the cabin crew, so maybe time for a name change and give the flag carrier status to a proper low-cost - and sadly the staff service has declined year on year. Onboard, the catering remains poor and surprised that Do&Co can produce such rubbish meals when I have sampled their Turkish Airlines equivalent in August that was superb - guessing BA are too tight with their budget. Onboard WiFi service is a joke and should be avoided.",ba standard continue decline every time fly time 45 min late departure seems norm ba information rude clueless boarding gate staff club lounge overcrowded dirty grubby cleaning staff could cope many cutback ba going apply become truly low cost 35 year veteran user ba people complained alex cruz cutting cost past 3 year seen decline quickly latest ceo one read occasional article ba improving thing yet see anything substantive seem british staff left amongst cabin crew maybe time name change give flag carrier status proper lowcost sadly staff service declined year year onboard catering remains poor surprised doco produce rubbish meal sampled turkish airline equivalent august superb guessing ba tight budget onboard wifi service joke avoided
2,"Awful. Business class check in queue just as long as for economy, probably because half of the desks were not staffed and Terminal 5 is chaotic. Business lounge overfull, couldn't get seats. Delayed flight, unmanaged chaos at the gate. No drink on boarding, first drink served an hour after take-off. The meal was poor, used to get better in economy on other airlines. British Airways has won the race to the bottom of the cheapskate stakes. Not looking forward to the flight home.",awful business class check queue long economy probably half desk staffed terminal 5 chaotic business lounge overfull couldnt get seat delayed flight unmanaged chaos gate drink boarding first drink served hour takeoff meal poor used get better economy airline british airway race bottom cheapskate stake looking forward flight home
3,"Not a reliable airline. You cannot trust the timing at all. I had a 6.45am flight, boarding was on time, but then we are sitting in the plane for already (1.5 hours) and the expected departure is still not 

the aim of that code is to apply several cleaning steps, started by removing the irrelevant characters liks symbols, etc, then lowercase to convert all characters into lower ones, then removing any punctuations, stop words like and, etc. then lemmatization for each word to convert each word to its origin, then I define the clean_text function to combine it into one cleaning pipeline.


Note: I added nltk.downlaod('stopwords') here not in the import libraries at the beginning cause it didnt run when i added there so i had to keep it inside the code itself.

# Feauture Engineering

In [ ]:
tfidf_vectorizer = TfidfVectorizer(max_features=500)
X = tfidf_vectorizer.fit_transform(df['cleaned_text'])
print(tfidf_vectorizer.get_feature_names_out())

['10' '12' '15' '20' '30' '40' '747' '777' '787' 'a380' 'able'
 'absolutely' 'access' 'across' 'actually' 'age' 'agent' 'ago' 'ahead'
 'air' 'aircraft' 'airline' 'airport' 'airway' 'aisle' 'almost' 'already'
 'also' 'although' 'always' 'announcement' 'another' 'anything' 'area'
 'around' 'arrival' 'arrived' 'ask' 'asked' 'attendant' 'attentive'
 'available' 'average' 'avios' 'avoid' 'away' 'awful' 'ba' 'back' 'bad'
 'bag' 'baggage' 'bar' 'bed' 'beef' 'behind' 'best' 'better' 'beverage'
 'big' 'bit' 'board' 'boarded' 'boarding' 'boeing' 'book' 'booked'
 'booking' 'bottle' 'breakfast' 'british' 'broken' 'budget' 'bus'
 'business' 'busy' 'buy' 'cabin' 'call' 'called' 'came' 'cancelled' 'cant'
 'captain' 'card' 'care' 'carrier' 'case' 'catering' 'champagne' 'change'
 'changed' 'charge' 'cheap' 'check' 'checked' 'checkin' 'cheese' 'chicken'
 'child' 'choice' 'choose' 'class' 'clean' 'clearly' 'club' 'coffee'
 'cold' 'come' 'comfort' 'comfortable' 'compared' 'complaint'
 'configuration' 'con

I used TF-IDF vectorizer to convert the text data into numerical one, and to reduce the common words, then I applied it on the new column (cleaned_text), then I printed the most important 500 words in the text.

# Bigram Model Training

In [ ]:
sentences = [text for text in df['cleaned_text']]
random.shuffle(sentences)
split_index = int(0.8 * len(sentences))
train_sentences = sentences[:split_index]
test_sentences = sentences[split_index:]

I extracted to cleaned text column that already has been preprocessed, then shuffle the list of sentences randomly, then split the data as 80% for training and 20% for testing

# Train the bigram model

In [ ]:
train_tokens = [list(nltk.word_tokenize(sentence)) for sentence in train_sentences]

train_data_bigram, vocab_bigram = nltk.lm.preprocessing.padded_everygram_pipeline(2, train_tokens)
bigram_model = nltk.lm.models.Laplace(2)
bigram_model.fit(train_data_bigram, vocab_bigram)

test_tokens = [list(nltk.word_tokenize(sentence)) for sentence in test_sentences]

test_data_bigram, _ = nltk.lm.preprocessing.padded_everygram_pipeline(2, test_tokens)
test_data_bigram = [list(everygrams) for everygrams in test_data_bigram]

I started by converting the training sentences into bigrams(to break down the sentences) and padding the sentences(to let the word do not have a following word) at both ends.

# Perplexity Calculation

In [ ]:
preplexity_bigram = bigram_model.perplexity(test_data_bigram)

print(f"Perplexity of Bigram Model on Test Data: {int(preplexity_bigram)}")

Perplexity of Bigram Model on Test Data: 8127


Calculate the perplexity to measure how well the model will predict the test data

In [ ]:
log_likelihood = bigram_model.score(test_data_bigram)
print(f"Log-likelihood of Bigram Model: {log_likelihood}")

Log-likelihood of Bigram Model: 9.286861876503311e-06


Evaluate the probability of the test under the trained model

# Dataset Splitting Train-Test

In [ ]:
vectorizer = TfidfVectorizer(max_features=500)
x = df["text"]
y = df["label"]

x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(x, y, test_size=0.2, random_state=42)

x_train = vectorizer.fit_transform(x_train).toarray()
x_test = vectorizer.transform(x_test).toarray()
print ("new_x_train:", x_train.shape)
print ("new_x_test:", x_test.shape)

new_x_train: (1058, 500)
new_x_test: (265, 500)


I extracted text and label columns and assigned it to x and y, as x will be the feature and y will be my target. then I splitted the data into training(80%) and testing(20%). then I applied the TF-IDF transformation into the text data. then I used toarray function to convert the sparse matrix into dense array, so I avoided that as the sparse matrix only storage non zeros values. then I printed x and y after the splitting.

# Model Training - Random Forest

In [ ]:
parameters_grid = {
    "criterion": ["gini", "entropy"],
    "n_estimators": range(200, 310, 30)
}
model = sklearn.model_selection.GridSearchCV(sklearn.ensemble.RandomForestClassifier(),
                                             parameters_grid, scoring="accuracy", cv=5, n_jobs=-1)
model. fit(x_train, y_train)
print ("Accuracy of best Random Forest classfier = {:.2f}".format(model.best_score_))
print("Best found hyperparameters of Random Forest classfier = )".format(model.best_params_))

Accuracy of best Random Forest classfier = 0.86
Best found hyperparameters of Random Forest classfier = )


I used the traditional machine learning model "RandomForest" as its a text dataset, I started by defining the hyperparameters, then GridsearchCV to choose the best combination of the hyperparameters. then the accuracy to check the maximize one, cv is 5 to split it into 5 parts so the model can be trained 5 times. then (model.fit) to train the random forest model on the training data. then I printed to check the accuracy.

# Model Evaluation

In [ ]:
y_predicted = model.predict(x_test)
accuracy = sklearn.metrics.accuracy_score(y_test, y_predicted)
print ("Accuracy = {:.2f}".format(accuracy))

Accuracy = 0.81


I trained the model to make predictions on the test data, the compare the predicted labels with the true labels.

In [ ]:
accuracy = accuracy_score(y_test, y_predicted)
precision = precision_score(y_test, y_predicted)
recall = recall_score(y_test, y_predicted)
f1 = f1_score(y_test, y_predicted)

print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

Accuracy: 0.81
Precision: 0.80
Recall: 0.71
F1 Score: 0.75


Here to check the performance of the model, the precision to check how many of the predicted (yes) are really yes, then the recall to check how many actual yes are corectly identified, the last one F1 score is a balanced one that consider both(precision and recall).
And I did in that way so it will be easy to create a comparison after i create another model, so I will be able to check the comparison in 4 parts (accuracy- precision- recall- f1)

# Transformer Model Training (DistilRoBERTa)

In [ ]:
model_name = "distilroberta-base"
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)

df_train, df_test = train_test_split(df, test_size=0.2)
df_train['encoded_labels'] = label_encoder.fit_transform(df_train['label'])
df_test['encoded_labels'] = label_encoder.transform(df_test['label'])

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)

train_dataset = datasets.Dataset.from_pandas(df_train)
test_dataset = datasets.Dataset.from_pandas(df_test)

train_dataset = train_dataset.map(preprocess_function, batched=True)
test_dataset = test_dataset.map(preprocess_function, batched=True)

train_dataset = train_dataset.rename_column("encoded_labels", "labels")
test_dataset = test_dataset.rename_column("encoded_labels", "labels")

data_collator = transformers.DataCollatorWithPadding(tokenizer=tokenizer)

model = transformers.AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(df['label'].unique()))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Map:   0%|          | 0/1058 [00:00<?, ? examples/s]

Map:   0%|          | 0/265 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/331M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


The model name "distilroberta-base" to define the pre trained model as its a smaller version of roberta, (the tokenizerr is intialized to convert the text to tokens Ids) then I splitted the data into training and testing.
labels here are encoded as its converted to numerical values.

then I applied the preprcess function to each example in the dataset,  (512 tokens as length cause this is the maximum in distilrobera so i cant 513 for example).
then i used the data collator to add padding to the input sequences so all the sentences in a batch are the same length.

note: I got inspired from a code on the internet and i added the reference in reference part.

# Define training arguments

In [ ]:
training_args = transformers.TrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=100,
    save_steps=500,
    load_best_model_at_end=True,
    fp16=True,
)

trainer = transformers.Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-38-526d962943b4>:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = transformers.Trainer(


The training arguments define how the training here should be conducted including the strategy, batch size, number of epochs, weight decay, logging, etc.
then the trainer intializes the trainer with the model, training arguments, datasets, tokenizer(to encode the text and the data collactor to handle the batching and padding .

# Train the model

In [ ]:
import wandb
wandb.init(project="British Airline Review Dataset")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


I imported wandb cause when i train the model it wasn't working and need an API key, so icreated an account.

In [ ]:
trainer.train()
trainer.save_model("./results" + model_name)

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss,Validation Loss
100,0.612700,0.647366
200,0.469100,0.553624


Started with the actual training process then the training model name with the results.

# Evaluate the model

In [ ]:
eval_results = trainer.evaluate()
print(eval_results)

{'eval_loss': 0.45323479175567627, 'eval_runtime': 0.8223, 'eval_samples_per_second': 322.26, 'eval_steps_per_second': 81.477, 'epoch': 1.0}


To evaluate the loop where the model is evaluated on the tested dataset.

# Predictions

In [ ]:
predictions, labels, _ = trainer.predict(test_dataset)
predicted_labels = predictions.argmax(axis=-1)

accuracy = accuracy_score(labels, predicted_labels)
precision = precision_score(labels, predicted_labels)
recall = recall_score(labels, predicted_labels)
f1 = f1_score(labels, predicted_labels)

print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

Accuracy: 0.89
Precision: 0.89
Recall: 0.86
F1 Score: 0.88


I used the trainer object to get predictions for testing, which returns 2 values: the prediction and the labels. then axis =-1 to convert the predicted probabilities into labels. then check the metrics( accuracy- precision- recall, f1 score) as I did in random forest model.

# Comparison and Visualization

In [ ]:
rf_accuracy = accuracy_score(y_test, y_predicted)
rf_precision = precision_score(y_test, y_predicted)
rf_recall = recall_score(y_test, y_predicted)
rf_f1 = f1_score(y_test, y_predicted)

distilroberta_accuracy = accuracy_score(labels, predicted_labels)
distilroberta_precision = precision_score(labels, predicted_labels)
distilroberta_recall = recall_score(labels, predicted_labels)
distilroberta_f1 = f1_score(labels, predicted_labels)

comparison_df = pd.DataFrame({
    "Model": ["Model 1 (Traditional)", "Model 2 (DistilRoBERTa)"],
    "Accuracy": [rf_accuracy, distilroberta_accuracy],
    "Precision": [rf_precision, distilroberta_precision],
    "Recall": [rf_recall, distilroberta_recall],
    "F1 Score": [rf_f1, distilroberta_f1]
})

print(comparison_df)

                     Model  Accuracy  Precision    Recall  F1 Score
0    Model 1 (Traditional)  0.811321   0.800000  0.710280  0.752475
1  Model 2 (DistilRoBERTa)  0.894340   0.894737  0.864407  0.879310


Here's the comparison between the best 2 models the traditional one(random forest) and the deep learning one (distilrobera) to check which one has the best accuracy, precision, recall and F1 score.

In [ ]:
metrics = ["Accuracy", "Precision", "Recall", "F1 Score"]
rf_scores = [rf_accuracy, rf_precision, rf_recall, rf_f1]
distilroberta_scores = [distilroberta_accuracy, distilroberta_precision, distilroberta_recall, distilroberta_f1]

x = np.arange(len(metrics))
width = 0.35

fig, ax = plt.subplots(figsize=(12, 6))
rects1 = ax.bar(x - width/2, rf_scores, width, label='Model 1 (Traditional)')
rects2 = ax.bar(x + width/2, distilroberta_scores, width, label='Model 2 (DistilRoBERTa)')

ax.set_xlabel('Metrics')
ax.set_ylabel('Scores')
ax.set_title('Model Comparison')
ax.set_xticks(x)
ax.set_xticklabels(metrics)
ax.legend()

def add_labels(rects):
    for rect in rects:
        height = rect.get_height()
        ax.annotate(f'{height:.2f}',
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),
                    textcoords="offset points",
                    ha='center', va='bottom')

add_labels(rects1)
add_labels(rects2)

plt.tight_layout()
plt.show()

NameError: name 'rf_accuracy' is not defined

I wanted to visualize the results to make it seen and easier to see the difference between the 2 models.
I started by defining the metrics that will be comparised in my 2 models which are (accuracy- precision- Recall- F1 score)
then specify the width for each bar in the bar chart.
then i added the needed size for width and height for the bar chart itself.
then added my labels(metrics) and the title for the bar chart.
then i defined the labels for the chart (the numerical ones for each bar)
then i displayed the plot.

# Conclusion

So at the end, here are some key points that I got it after the anyalsis.
The deep learning model (distilroberta model) acheived the highest accuracy among the 3 models.
The traditional model (Random forest) also achieved a high accuracy which is 82% but the other model have the highest rate, so I would recommend it.

# References

https://yzsam.com/html/binuWt.html

https://discuss.huggingface.co/t/solution-for-graph-to-multiple-sentences-paragraph-generation/33826/2